In [ ]:
# importação de modulos
import pandas as pd
from funcoes import funcoes

In [ ]:
# instanciação da classe funcoes
func = funcoes(
      servidor=f'DESKTOP-NRPKK10\SQLEXPRESS'
    , banco=f'analise_dados'
    , driver=f'ODBC Driver 17 for SQL Server'
)

In [ ]:
# parametro lines=True por conta do \n no final das linhas do json
conteudo_arquivo = pd.read_json(f'D:\projetos_luis\Gift_Cards.json', lines=True)

In [ ]:
# renomeia colunas
df_cards = conteudo_arquivo.rename(
    columns={
          'overall':'nota'
        , 'vote':'votos'
        , 'verified':'user_verificado'
        , 'reviewTime':'data_avaliacao'
        , 'reviewerID':'id_avaliacao'
        , 'asin':'id_produto'
        , 'style':'valor'
        , 'reviewerName':'nome_usuario'
        , 'reviewText':'texto'
        , 'summary':'titulo'
        , 'unixReviewTime':'data_unix'
        , 'image':'imagem'
    }
)

In [ ]:
# criação dataframe vazio
df_ajustado = pd.DataFrame()

In [ ]:
# normalizando colunas e inserindo em um novo dataframe
df_ajustado['user_verificado'] = df_cards['user_verificado'].astype(int)
df_ajustado['data_avaliacao'] = pd.to_datetime(df_cards['data_avaliacao'], format='%m %d, %Y').dt.strftime('%d-%m-%Y')
df_ajustado['valor'] = df_cards['valor'].apply(func.extrair_numero).fillna(0).astype(int)
df_ajustado['nota'] = df_cards['nota'].astype(int)
df_ajustado['votos'] = df_cards['votos'].fillna(0).replace(r',', r'.', regex=True).astype(float).astype(int)
df_ajustado['id_avaliacao'] = df_cards['id_avaliacao']
df_ajustado['id_produto'] = df_cards['id_produto']
df_ajustado['nome_usuario'] = df_cards['nome_usuario']
df_ajustado['texto'] = df_cards['texto']
df_ajustado['titulo'] = df_cards['titulo']

In [ ]:
# carrega os dados no banco utilizando a função da classe funcoes
func.subir_dados(
      df=df_ajustado
    , tabela='amazon_gift_cards'
)

In [ ]:
func.query_sql(f'EXEC DBO.PRC_AMAZON_PALAVRAS')

In [ ]:
func.query_sql(f'EXEC DBO.PRC_AMAZON_AGG')

In [ ]:
func.fechar_conexao()